In [ ]:
using Revise

In [ ]:
using RigidBodyDynamics
using RigidBodyDynamics.PDControl
using RigidBodyDynamics.Contact
const rbd = RigidBodyDynamics
using MeshCatMechanisms
using MeshCat
using Statistics
using LightXML
using StaticArrays
using Optim
using LinearAlgebra
using RigidBodySim

In [ ]:
vis = Visualizer()

In [ ]:
open(vis)

In [ ]:
"""
Parse the Drake-specific `loop_joint` tags from a URDF and use them
to add non-tree joints to the given mechanism.
"""
function add_loop_joints!(mechanism::Mechanism{T}, urdf::AbstractString) where T
    doc = parse_file(urdf)
    xml_root = LightXML.root(doc)
    xml_loops = get_elements_by_tagname(xml_root, "loop_joint")
    for xml_loop in xml_loops
        name = attribute(xml_loop, "name")
        @assert attribute(xml_loop, "type") == "continuous"
        axis = SVector{3}(rbd.parse_vector(T, find_element(xml_loop, "axis"), "xyz", "1 0 0"))
        joint = Joint(name, Revolute(axis))
        xml_link1 = find_element(xml_loop, "link1")
        body1 = findbody(mechanism, attribute(xml_link1, "link"))
        H1 = Transform3D(frame_before(joint), default_frame(body1),
            rbd.parse_pose(T, xml_link1)...)
        xml_link2 = find_element(xml_loop, "link2")
        body2 = findbody(mechanism, attribute(xml_link2, "link"))
        H2 = Transform3D(frame_after(joint), default_frame(body2),
            rbd.parse_pose(T, xml_link2)...)
        attach!(mechanism, body1, body2, joint,
            joint_pose = H1,
            successor_pose = inv(H2))
    end
end         

In [ ]:
"""
Return a function which maps a configuration vector
for the given mechanism to the total squared error in
the alignment of the robot's non-tree joints.
"""
function loop_joint_error(robot)
    let statecache = StateCache(robot), resultcache = DynamicsResultCache(robot)
        function(q)
            state = statecache[eltype(q)]
            result = resultcache[eltype(q)]
            set_configuration!(state, q)
            zero_velocity!(state)
            gains = rbd.CustomCollections.ConstDict{JointID}(SE3PDGains(PDGains(1, 0), PDGains(1, 0)))
            rbd.constraint_bias!(result, state; stabilization_gains=gains)
            ret = result.constraintbias ⋅ result.constraintbias
        end
    end
end

In [ ]:
urdf = "data/Strandbeest.urdf"
robot = parse_urdf(Float64, urdf)
add_loop_joints!(robot, urdf)

crossbar = findbody(robot, "crossbar")
basejoint = joint_to_parent(crossbar, robot)
floatingjoint = Joint(basejoint.name, frame_before(basejoint),
    frame_after(basejoint), QuaternionFloating{Float64}())
replace_joint!(robot, basejoint, floatingjoint)


In [ ]:
# Find an initial state of the robot which correctly aligns all 
# the loop joints.

# We do this before adding the contact points to work around 
# https://github.com/JuliaRobotics/RigidBodyDynamics.jl/issues/483

# First, initialize the state with a configuration within the 
# joint limits:
bounds = collect(Iterators.flatten(rbd.position_bounds.(tree_joints(robot))))
lb = rbd.lower.(bounds)
ub = rbd.upper.(bounds);
state = MechanismState(robot)
set_configuration!(state, clamp.(configuration(state), lb, ub))

# Use Optim's Fminbox to minimize the loop joint error within
# the joint limits:
cost = loop_joint_error(robot)
result = Optim.optimize(cost, lb, ub, 
    Vector(configuration(state)), 
    Fminbox(LBFGS()), autodiff=:forward)

# Verify that we've actually closed all the loops
@assert Optim.minimum(result) < 1e-9

q0 = Optim.minimizer(result);

In [ ]:
contactmodel = SoftContactModel(hunt_crossley_hertz(k = 500e3), ViscoelasticCoulombModel(1.0, 20e3, 100.))
for body in bodies(robot)
    if occursin("bars_g_h_i", string(body))
        frame = default_frame(body)
        point = Point3D(frame, 0, 0, 0.49)
        add_contact_point!(body, ContactPoint(
            point, contactmodel))
        
    end
end
add_environment_primitive!(robot, 
HalfSpace3D(Point3D(root_frame(robot), 0., 0, 0),
        FreeVector3D(root_frame(robot), normalize(SVector(1., 0, 5)))))

state = MechanismState(robot)
set_configuration!(state, q0)
mvis = MechanismVisualizer(robot, URDFVisuals(urdf), vis)
copyto!(mvis, state)

In [ ]:
set_configuration!(state, q0)
q = configuration(state)
r = Quat(RotY(atan(1/5)))
q[1:4] .= (r.w, r.x, r.y, r.z)
q[7] = 1.1
set_configuration!(state, q)
copyto!(mvis, state)

In [ ]:
using GeometryTypes
using CoordinateTransformations
using Colors

In [ ]:
setobject!(vis[:ground], HyperRectangle(Vec(-50, -50, -0.01), Vec(100, 100, 0.02)), 
MeshPhongMaterial(color=RGB(0.5, 0.5, 0.5)))
settransform!(vis[:ground], LinearMap(RotY(atan(1/5))))

In [ ]:
# Simulate the robot
dynamics = Dynamics(robot)
problem = ODEProblem(dynamics, state, (0, 10.0))
@time solution = solve(problem, Tsit5(), abs_tol=1e-6, dt=1e-6)
setanimation!(mvis, solution)